<a href="https://colab.research.google.com/github/CalvHobbes/shop_india/blob/main/smolagents/price_comp_smolagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install smolagents smolagents[litellm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1
  Attempting uninstall: httpx

In [2]:
from google.colab import userdata
import os

openapi_key = userdata.get('OPENAPI_KEY')
hf_token = userdata.get('HF_TOKEN')

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py

--2025-01-31 13:34:30--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1817 (1.8K) [text/plain]
Saving to: ‘search_Croma.py’

search_Croma.py     100%[===================>]   1.77K  --.-KB/s    in 0s      

2025-01-31 13:34:30 (28.2 MB/s) - ‘search_Croma.py’ saved [1817/1817]

--2025-01-31 13:34:31--  https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1466

In [5]:
model_id = "gpt-4o-mini"

In [6]:
from smolagents import CodeAgent, LiteLLMModel

model = LiteLLMModel(model_id=model_id, api_key=openapi_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [7]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [8]:
from smolagents import tool
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance

@tool
def search_croma_tool(product: str) -> str:
    """
    This is a tool that searches for the user specified product on Croma and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_croma(product)


@tool
def search_reliance_tool(product: str) -> str:
    """
    This is a tool that searches for the user specified product on Reliance and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_reliance(product)


In [9]:
from smolagents.prompts import CODE_SYSTEM_PROMPT
modified_system_prompt = f"{CODE_SYSTEM_PROMPT}\n{instructions}"
agent = CodeAgent(tools=[search_croma_tool,search_reliance_tool], model=model, system_prompt=modified_system_prompt, additional_authorized_imports=['requests', 'bs4', 'json'])
agent.run("find best price for iphone 16 pro max")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ find best price for iphone 16 pro max                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  croma_result = search_croma_tool(product="iPhone 16 Pro Max")                                                    
  reliance_result = search_reliance_tool(product="iPhone 16 Pro Max")                                              
                                                                                                                   
  print("Croma Result:", croma_result)                                                                             
  print("Reliance Result:", reliance_result)                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Croma Result: [{'name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'price': 137900.0}, {'name': 
'Apple iPhone 16 Pro Max (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'price': 137900.0}, {'name': 'Apple
iPhone 16 Pro Max (1TB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-natural-titanium-/p/309774', 'price': 177900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro Max (512GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-natural-titanium-/p/309751', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (512GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-desert-titanium-/p/309750', 'price': 157900.0}, {'name': 
'Apple iPhone 16 Pro Max (1TB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-white-titanium-/p/309771', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (1TB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-desert-titanium-/p/309773', 'price': 177900.0}, {'name': 'Apple 
iPhone 16 Pro Max (512GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-white-titanium-/p/309749', 'price': 157900.0}, {'name': 'Apple
iPhone 16 Pro (256GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-desert-titanium-/p/309731', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Desert Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-desert-titanium-/p/309727', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-black-titanium-/p/309729', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-natural-titanium-/p/309732', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Natural Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-natural-titanium-/p/309728', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-black-titanium-/p/309725', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (256GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-256gb-white-titanium-/p/309730', 'price': 122900.0}, {'name': 'Apple 
iPhone 16 Pro (128GB, White Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-128gb-white-titanium-/p/309726', 'price': 112900.0}, {'name': 'Apple 
iPhone 16 Pro (512GB, Black Titanium)', 'url': 
'https://www.croma.com/apple-iphone-16-pro-512gb-black-titanium-/p/309733', 'price': 142900.0}]
Reliance Result: [{'name': 'Apple iPhone 16 Pro Max 1 TB, Desert Titanium', 'url': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-1-tb-desert-titanium/p/494423069', 'price': '1,77,900.00'},
{'name': 'Apple iPhone 16 Pro Max 256 GB, Black Titanium', 'url': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059', 'price': 
'1,37,900.00'}, {'name': 'Apple iPhone 16 Pro Max 512 GB, White Titanium', 'url': 
'https://www.reliancedigital.in/

[Step 0: Duration 4.18 seconds| Input tokens: 2,416 | Output tokens: 109]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Best Prices": [                                                                                             
          {                                                                                                        
              "Product Name": "Apple iPhone 16 Pro Max (256GB, Desert Titanium)",                                  
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746",      
              "Price": 137900.0                                                                                    
          },                                                                                                       
          {                                                                                                        
              "Product Name": "Apple iPhone 16 Pro Max (256GB, Black Titanium)",                                   
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742",       
              "Price": 137900.0                                                                                    
          },                                                                                                       
          {                                                                                                        
              "Product Name": "Apple iPhone 16 Pro Max (256GB, Natural Titanium)",                                 
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747",     
              "Price": 137900.0                                                                                    
          },                                                                                                       
          {                                                                                                        
              "Product Name": "Apple iPhone 16 Pro Max 256 GB, Black Titanium",                                    
              "Store": "Reliance",                                                                                 
              "Product URL":                                                                                       
  "https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059",                      
              "Price": 137900.0                                                                                    
          },                                                                                                       
          {                                                                                                        
              "Product Name": "Apple iPhone 16 Pro Max (512GB, Black Titanium)",                                   
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748",       
              "Price": 157900.0                                                                                    
          },                                                                                                       
          {                                              

Out - Final answer: {'Best Prices': [{'Product Name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)', 'Store': 
'Croma', 'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'Price': 
137900.0}, {'Product Name': 'Apple iPhone 16 Pro Max (256GB, Black Titanium)', 'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'Price': 137900.0}, {'Product 
Name': 'Apple iPhone 16 Pro Max (256GB, Natural Titanium)', 'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'Price': 137900.0}, {'Product 
Name': 'Apple iPhone 16 Pro Max 256 GB, Black Titanium', 'Store': 'Reliance', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059', 'Price': 137900.0}, 
{'Product Name': 'Apple iPhone 16 Pro Max (512GB, Black Titanium)', 'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748', 'Price': 157900.0}, {'Product 
Name': 'Apple iPhone 16 Pro Max 512 GB, Desert Titanium', 'Store': 'Reliance', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-desert-titanium/p/494423065', 'Price': 157900.0}, 
{'Product Name': 'Apple iPhone 16 Pro Max (1TB, Black Titanium)', 'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-1tb-black-titanium-/p/309754', 'Price': 177900.0}]}

[Step 3: Duration 13.58 seconds| Input tokens: 29,730 | Output tokens: 2,941]

{'Best Prices': [{'Product Name': 'Apple iPhone 16 Pro Max (256GB, Desert Titanium)',
   'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746',
   'Price': 137900.0},
  {'Product Name': 'Apple iPhone 16 Pro Max (256GB, Black Titanium)',
   'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742',
   'Price': 137900.0},
  {'Product Name': 'Apple iPhone 16 Pro Max (256GB, Natural Titanium)',
   'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747',
   'Price': 137900.0},
  {'Product Name': 'Apple iPhone 16 Pro Max 256 GB, Black Titanium',
   'Store': 'Reliance',
   'Product URL': 'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059',
   'Price': 137900.0},
  {'Product Name': 'Apple iPhone 16 Pro Max (512GB, Black Titanium)',
   'Store': 'Croma',
   'Product URL': 'https:/

In [10]:
# Using multiple agents
from smolagents import ManagedAgent

croma_agent = CodeAgent(tools=[search_croma_tool], model=model)

managed_croma_agent = ManagedAgent(
    agent=croma_agent,
    name="croma_search",
    description="Search for products on Croma site. Give it the product to search for as an argument"
)

reliance_agent = CodeAgent(tools=[search_reliance_tool], model=model)

managed_reliance_agent = ManagedAgent(
    agent=reliance_agent,
    name="reliance_search",
    description="Search for products on Reliance Digital site. Give it the product to search for as an argument"
)



In [11]:
price_comp_agent = CodeAgent(
    tools=[], model=model, managed_agents=[managed_croma_agent,managed_reliance_agent],
    system_prompt=modified_system_prompt, additional_authorized_imports=['requests', 'bs4', 'json']
)

print(price_comp_agent.system_prompt)

You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool.

Here are a few examples using n

In [12]:
price_comp_agent.run("what is the cheapest price for iphone 16 pro max?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ what is the cheapest price for iphone 16 pro max?                                                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  croma_results = croma_search("iPhone 16 Pro Max")                                                                
  reliance_results = reliance_search("iPhone 16 Pro Max")                                                          
  print("Croma Results:", croma_results)                                                                           
  print("Reliance Results:", reliance_results)                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'croma_search'.                                                                    │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ iPhone 16 Pro Max                                                                                               │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Product Name": "iPhone 16 Pro Max",                                                                         
      "Store Details": [                                                                                           
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746",      
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747",     
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742",       
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Croma",                                                                                    
              "Product URL": "https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744",       
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Reliance Digital",                                                                         
              "Product URL":                                                                                       
  "https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059",                      
              "Price": "₹137900"                                                                                   
          },                                                                                                       
          {                                                                                                        
              "Store": "Reliance Digital",                                                                         
              "Product URL":                                                                                       
  "https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060",                      
              "Price": "₹137900"                         

Out - Final answer: {'Product Name': 'iPhone 16 Pro Max', 'Store Details': [{'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746', 'Price': '₹137900'}, {'Store': 
'Croma', 'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747', 'Price': 
'₹137900'}, {'Store': 'Croma', 'Product URL': 
'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742', 'Price': '₹137900'}, {'Store': 
'Croma', 'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744', 'Price': 
'₹137900'}, {'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059', 'Price': '₹137900'}, 
{'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060', 'Price': '₹137900'}, 
{'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-desert-titanium/p/494423061', 'Price': '₹137900'}, 
{'Store': 'Reliance Digital', 'Product URL': 
'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-natural-titanium/p/494423062', 'Price': '₹137900'}]}

[Step 2: Duration 11.69 seconds| Input tokens: 13,484 | Output tokens: 2,299]

{'Product Name': 'iPhone 16 Pro Max',
 'Store Details': [{'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746',
   'Price': '₹137900'},
  {'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-natural-titanium-/p/309747',
   'Price': '₹137900'},
  {'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742',
   'Price': '₹137900'},
  {'Store': 'Croma',
   'Product URL': 'https://www.croma.com/apple-iphone-16-pro-max-256gb-white-titanium-/p/309744',
   'Price': '₹137900'},
  {'Store': 'Reliance Digital',
   'Product URL': 'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059',
   'Price': '₹137900'},
  {'Store': 'Reliance Digital',
   'Product URL': 'https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060',
   'Price': '₹137900'},
  {'Store': 'Reliance Digital',
   'Pr